# Hyperparameter Tuning using HyperDrive

In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import azureml.core
import tensorflow
print("SDK version:", azureml.core.VERSION)
print("Tensorflow version:", tensorflow.__version__)

2023-09-04 10:34:47.273465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 10:35:02.928763: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-04 10:35:02.929013: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-09-04 10:35:02.929032: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

SDK version: 1.51.0
Tensorflow version: 2.11.0


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment_name = 'mnist-hyperdrive-experiment'
exp = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: quick-starts-ws-241000
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-241000


## Attach an Compute Instance

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Udacity subscription does not allow to create GPU instance
cpu_instance = "mnist-instance"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_instance)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS2_V2",
                                                           vm_priority='lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_instance, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Dataset

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. The digits have been size-normalized and centered in a fixed-size image. The images were centered in a 28x28 image by computing the center of mass of the pixels. The data is stored in a very simple file format designed for storing vectors and multidimensional matrices.

Reference: http://yann.lecun.com/exdb/mnist/

### Download the Data

In [4]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print("Train image shape: {}", train_images.shape)
print("Train label shape: {}", train_labels.shape)

11490434/11490434 [==============================] - 0s 0us/step
Train image shape: {} (60000, 28, 28)
Train label shape: {} (60000,)


### Transform the Data

In [5]:
import pandas as pd

flattened_train_image = train_images.reshape(train_images.shape[0], -1)
flattened_test_image = test_images.reshape(test_images.shape[0], -1)

train_images_df = pd.DataFrame(flattened_train_image)
train_labels_series = pd.Series(train_labels, name="labels")
train_df = pd.concat([train_images_df, train_labels_series], axis=1)

test_images_df = pd.DataFrame(flattened_test_image)
test_labels_series = pd.Series(test_labels, name="labels")
test_df = pd.concat([test_images_df, test_labels_series], axis=1)

print(train_df.head())
print(test_df.head())

   0  1  2  3  4  5  6  7  8  9  ...  775  776  777  778  779  780  781  782  \
0  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
1  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
2  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
3  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
4  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   

   783  labels  
0    0       5  
1    0       0  
2    0       4  
3    0       1  
4    0       9  

[5 rows x 785 columns]
   0  1  2  3  4  5  6  7  8  9  ...  775  776  777  778  779  780  781  782  \
0  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
1  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
2  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
3  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0  

### Register the Data

In [6]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
Dataset.Tabular.register_pandas_dataframe(dataframe=train_df, name='mnist-train', target=datastore)
Dataset.Tabular.register_pandas_dataframe(dataframe=test_df, name='mnist-test', target=datastore)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/d2428547-d267-455d-9139-ff747a9661df/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/d0aa4261-f218-4474-b681-294cb9114809/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/d0aa4261-f218-4474-b681-294cb9114809/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "b954fadb-adfc-4400-b0df-bcb39b8efbd6",
    "name": "mnist-test",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-241000', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-241000')"
  }
}

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
from azureml.core import ScriptRunConfig, Environment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, normal, uniform
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.policy import BanditPolicy

# # Experiment with from lower to higher dropout
# sampling = RandomParameterSampling({
#     "--dropout1": uniform(0.0, 0.1),
#     "--dropout2": uniform(0.2, 0.5),
#     "--dropout3": normal(0.3, 0.7),
#     "--epochs": choice(10, 20, 40)
# })

# # Experiment with from higher to lower dropout
# sampling = RandomParameterSampling({
#     "--dropout1": normal(0.3, 0.7),
#     "--dropout1": uniform(0.0, 0.1),
#     "--dropout3": uniform(0.2, 0.5),
#     "--epochs": choice(10, 20, 40)
# })

# Experiment with the full range of possibilites
sampling = RandomParameterSampling({
    "--dropout1": uniform(0.0, 0.5),
    "--dropout2": uniform(0.0, 0.5),
    "--epochs": choice(10, 20, 40)
})

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)
tensorflow_env = Environment.from_conda_specification(name='tensorflow-env', file_path='conda_dependencies.yml')
script = ScriptRunConfig(source_directory='.',
                         script='mnist_train.py',
                         compute_target=compute_target,
                         environment=tensorflow_env)

hyperdrive_config = HyperDriveConfig(run_config=script,
                    hyperparameter_sampling=sampling,
                    policy=policy,
                    primary_metric_name='accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=8,
                    max_concurrent_runs=4)

In [8]:
# Submit the hyperdrive run to the experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc
Web View: https://ml.azure.com/runs/HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-241000/workspaces/quick-starts-ws-241000&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-04T10:55:50.593934][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-09-04T10:55:51.1260909Z][SCHEDULER][INFO]Scheduling job, id='HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc_0' 
[2023-09-04T10:55:51.3333702Z][SCHEDULER][INFO]Scheduling job, id='HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc_1' 
[2023-09-04T10:55:51.3803313Z][SCHEDULER][INFO]Scheduling job, id='HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc_2' 
[2023-09-04T10:55:51.358841][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-09-04T10:55:51.5536040Z][SCHEDULER][INFO]Scheduling job, id='HD_3d7f1311-4982-4426-b8cc-b9ce1065b4

{'runId': 'HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc',
 'target': 'mnist-instance',
 'status': 'Completed',
 'startTimeUtc': '2023-09-04T10:55:49.832108Z',
 'endTimeUtc': '2023-09-04T11:36:54.852086Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9da854ea-2469-4011-9986-1ba32314383a',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'score': '0.9793999791145325',
  'best_child_run_id': 'HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryV

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(f"Best run: {best_run}\n")
print(f"Best run metrics: {best_run_metrics}\n")
print(f"Associated files: {best_run.get_file_names()}\n")

Best run: Run(Experiment: mnist-hyperdrive-experiment,
Id: HD_3d7f1311-4982-4426-b8cc-b9ce1065b4cc_0,
Type: azureml.scriptrun,
Status: Completed)

Best run metrics: {'First layer dropout value:': 0.11697766872072268, 'Number of Epoch:': 40, 'Second layer dropout value:': 0.38212205639235863, 'accuracy': 0.9793999791145325, 'training_time': 751.3384034633636}

Associated files: ['azureml-logs/20_image_build_log.txt', 'outputs/mnist_model.keras', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']



In [14]:
# TODO: Download the model
# Downloading model from the run into the workspace
best_run.download_file(name='outputs/mnist_model.keras', output_file_path='./mnist_model.keras')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
registered_model = best_run.register_model(
    model_name="mnist_hyperdrive",
    model_framework="TensorFlow",
    model_path="outputs/mnist_model.keras")

TODO: In the cell below, send a request to the web service you deployed to test it.

### Deploy an Azure Container Instance Web Service

In [28]:
Model.list(ws)

[Model(workspace=Workspace.create(name='quick-starts-ws-241000', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-241000'), name=mnist_hyperdrive, id=mnist_hyperdrive:2, version=2, tags={}, properties={}),
 Model(workspace=Workspace.create(name='quick-starts-ws-241000', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-241000'), name=mnist_hyperdrive, id=mnist_hyperdrive:1, version=1, tags={}, properties={}),
 Model(workspace=Workspace.create(name='quick-starts-ws-241000', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-241000'), name=mnist_model, id=mnist_model:1, version=1, tags={}, properties={})]

In [25]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice


# Custom configuration for the deployment
# tensorflow_env = Environment.from_conda_specification(name='tensorflow-env', file_path='conda_dependencies.yml')
# inference_config = InferenceConfig(entry_script='mnist_score.py', environment=tensorflow_env)
registered_model = best_run.register_model(model_name="mnist_hyperdrive", model_framework="TensorFlow", model_path="outputs/mnist_model.keras")
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the registered model as a web service endpoint
service_name = 'mnist-hyperdrive-service'
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[registered_model],
    # inference_config=inference_config,
    deployment_config=deployment_config)

# Wait for the deployment to complete
service.wait_for_deployment(show_output=True)
print(service.state)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


WebserviceException: WebserviceException:
	Message: Service mnist-hyperdrive-service with the same name already exists, please use a different service name or delete the existing service.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service mnist-hyperdrive-service with the same name already exists, please use a different service name or delete the existing service."
    }
}

### Test the Web Service Endpoint

In [ ]:
from utils import visualize
# TODO: Replace it with the consume script provided by AML studio
# Get input data (use the dumped json from the inference script) -> Show as img -> Inference
input_data = {
    "input_key": input_value
}

# Call the web service endpoint
result = service.run(input_data)
print("Web service response:", result)

# TODO: Get the predicted labels ID
predicted_labels = ...
# TODO: Get the test labels ID
test_labels = ...

visualize[result[0]]
print("Predicted labels: ", predicted_labels[0])
print("Ground truth: ", test_labels[0])

## Delete the Resource
TODO: In the cell below, print the logs of the web service and delete the service
- webservice
- cluster

In [ ]:
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

